In [9]:
import requests
import json

credentials = json.load(open('../keys/tmdb_key.json'))
token = credentials['token']


In [12]:

url = "https://api.themoviedb.org/3/account/20762066/watchlist"

payload = {
    "media_type": "movie",
    "media_id": 1104845,
    "watchlist": True
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": f"Bearer {token}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"success":true,"status_code":12,"status_message":"The item/record was updated successfully."}


In [ ]:

url = "https://api.themoviedb.org/3/authentication/token/new"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}"
}

response = requests.get(url, headers=headers)

print(response.text)

In [14]:
# URL for Validation is https://www.themoviedb.org/authenticate/{request_token}

# Get Session ID:
url = "https://api.themoviedb.org/3/authentication/session/new"

payload = { "request_token": "..." }
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": f"Bearer {token}"
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)